In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "UDdjs5w=6siS3emjl=Y5"),
    ca_certs="C:/Users/C4/Downloads/elasticsearch-8.16.1/config/certs/http_ca.crt"
)
print(es.ping()) 

True


## Prepare the data

In [3]:
import pandas as pd

df = pd.read_csv("research.csv")



In [4]:
df.isna().value_counts()

doc_id  Title  Abstract  DOI  
False   False  False     False    4291
               True      False      15
Name: count, dtype: int64

In [5]:
df.fillna("None", inplace=True)

## Convert the relevant field to Vector using BERT model

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\C4\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df["AbstractVector"] = df["Abstract"].apply(lambda x: model.encode(x))

In [15]:

df.head()

,doc_id,Title,Abstract,DOI,AbstractVector
0,1,[Light scattering by cell suspensions in norma...,The characteristics of light scattering of cel...,https://pubmed.ncbi.nlm.nih.gov/1100/,"[0.006501656, -0.14199008, 0.009908089, -0.053..."
1,2,[The fragmentation and reconstruction of the o...,The protein and proteolipid complexes and olig...,https://pubmed.ncbi.nlm.nih.gov/1103/,"[0.0033499182, -0.053660918, 0.053273335, 0.02..."
2,3,[Study of acetyl-CoA-synthetase from staphyloc...,Acetyl-CoA-synthetase was isolated from cells ...,https://pubmed.ncbi.nlm.nih.gov/1104/,"[0.011662829, -0.02032138, 0.018208493, -0.043..."
3,4,[Effect of prosthetic group of horseradish per...,Constants of inactivation rate of horseradish ...,https://pubmed.ncbi.nlm.nih.gov/1105/,"[-0.023887841, -0.033442575, 0.030742425, -0.0..."
4,5,[Wheat protein disulfide reductase],Proteindisulphide reductase is isolated and pa...,https://pubmed.ncbi.nlm.nih.gov/1106/,"[-0.042119388, -0.06871767, 0.014972381, 0.026..."


In [16]:
es.ping()

True

## Create new index in ElasticSearch!

In [18]:
from indexMapping import indexMapping
es.indices.create(index="is", body=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'is'})

## Ingest the data into index

In [11]:
record_list = df.to_dict("records")

In [19]:

for record in record_list:
    try:
        es.index(index="is", document=record, id=record['doc_id'])
    except Exception as e:
        print(e)

In [20]:
es.count(index="is")

ObjectApiResponse({'count': 4306, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data

In [23]:
input_keyword = "Infection"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "AbstractVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 10000, 
}

res = es.knn_search(index="is", knn=query , source=["Title","Abstract"])
res["hits"]["hits"]

C:\Users\C4\AppData\Local\Temp\ipykernel_940\676040731.py:11: GeneralAvailabilityWarning: This API is in technical preview and may be changed or removed in a future release. Elastic will work to fix any issues, but features in technical preview are not subject to the support SLA of official GA features.
  res = es.knn_search(index="is", knn=query , source=["Title","Abstract"])
C:\Users\C4\AppData\Local\Temp\ipykernel_940\676040731.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="is", knn=query , source=["Title","Abstract"])


[{'_index': 'is',
  '_id': '961',
  '_score': 0.42772973,
  '_source': {'Title': 'Macroscopic patterns of interacting contagions are indistinguishable from social reinforcement',
   'Abstract': 'From ‘fake news’ to innovative technologies, many contagions spread as complex contagions via a process of social reinforcement, where multiple exposures are distinct from prolonged exposure to a single source1. Contrarily, biological agents such as Ebola or measles are typically thought to spread as simple contagions2. Here, we demonstrate that these different spreading mechanisms can have indistinguishable population-level dynamics once multiple contagions interact. In the social context, our results highlight the challenge of identifying and quantifying spreading mechanisms, such as social reinforcement3, in a world where an innumerable number of ideas, memes and behaviours interact. In the biological context, this parallel allows the use of complex contagions to effectively quantify the non